<a href="https://colab.research.google.com/github/KangJaewan/New-Project/blob/main/%EC%BD%94%EB%9E%A9%ED%85%8C%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('hi')

hi


In [8]:
%%time

import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as f
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

CPU times: user 26 µs, sys: 4 µs, total: 30 µs
Wall time: 33.4 µs


In [13]:
%%time

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cifar10_train = CIFAR10('./data', train=True, download=True, transform=transforms.ToTensor())
cifar10_test = CIFAR10('./data', train=False, download=True, transform=transforms.ToTensor())

batch_size = 64
train_loader = DataLoader(cifar10_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(cifar10_test, batch_size=batch_size, shuffle=False)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
    def forward(self, x):
        x = self.pool(f.relu(self.conv1(x)))
        x = self.pool(f.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = f.relu(self.fc1(x))
        return x

net = Net().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

recode_loss_train = []
recode_loss_test = []

x_test, y_test = next(iter(test_loader))
x_test, y_test = x_test.to(device), y_test.to(device)

for i in range(10):
  net.train()
  loss_train = 0.0
  for j, (x_train, y_train) in enumerate(train_loader):
    x_train, y_train = x_train.to(device), y_train.to(device)
    y = net(x_train)
    loss = loss_fn(y, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  loss_train /= (j+1)
  recode_loss_train.append(loss_train)

  net.eval()
  y_pred = net(x_test)
  loss_test = loss_fn(y_pred, y_test)
  recode_loss_train.append(loss_test.item())

cpu
CPU times: user 3min 31s, sys: 217 ms, total: 3min 32s
Wall time: 3min 33s
